In [51]:
import pandas as pd

In [52]:
data = pd.read_csv("data/loan_data.csv")

In [53]:
data = data.drop("Unnamed: 0", axis=1)

In [54]:
data

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,39651438,NaN,32000.0,32000.0,32000.0,60 months,10.49,687.65,B,B3,Public Service,10+ years,MORTGAGE,120000.0,Verified,Feb-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,919xx,CA,24.05,0.0,Oct-1981,735.0,739.0,0.0,NaN,NaN,20.0,0.0,39687.0,57.8,42.0,w,7181.91,7181.91,33676.200000,33676.20,24818.09,8858.11,0.0,0.0,0.0,Mar-2019,687.65,Apr-2019,Mar-2019,794.0,790.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,457317.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82400.0,NaN,NaN,NaN,6.0,26901.0,31079.0,71.2,0.0,0.0,154.0,165.0,1.0,1.0,2.0,9.0,NaN,7.0,NaN,0.0,6.0,11.0,8.0,20.0,7.0,14.0,31.0,11.0,20.0,0.0,0.0,0.0,4.0,100.0,28.6,0.0,0.0,556496.0,103647.0,64100.0,72197.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,16411620,NaN,9600.0,9600.0,9600.0,36 months,12.99,323.42,C,C1,NaN,NaN,RENT,21900.0,Verified,May-2014,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,331xx,FL,10.03,0.0,Apr-2001,685.0,689.0,1.0,NaN,118.0,13.0,1.0,4509.0,38.9,20.0,w,0.00,0.00,11643.969042,11643.97,9600.00,2043.97,0.0,0.0,0.0,Jun-2017,0.85,NaN,Aug-2017,544.0,540.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,4509.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11600.0,NaN,NaN,NaN,6.0,347.0,871.0,63.7,0.0,0.0,97.0,157.0,1.0,1.0,0.0,26.0,NaN,1.0,NaN,0.0,2.0,5.0,2.0,5.0,2.0,13.0,18.0,5.0,13.0,0.0,0.0,0.0,2.0,100.0,50.0,1.0,0.0,11600.0,4509.0,2400.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

### Identify and Remove Leakage Features

In [55]:
data = data.drop(["total_pymnt", "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee", "last_pymnt_d", "last_pymnt_amnt", "recoveries", "collection_recovery_fee", "out_prncp", "out_prncp_inv", "funded_amnt", "funded_amnt_inv", "last_credit_pull_d", "last_fico_range_high", "last_fico_range_low", "acc_now_delinq", "delinq_amnt"], axis=1)

### Handle missing data.

**i) Drop columns with >20% missing values**


In [56]:
percentMissing = (data.isna().sum() / len(data)).sort_values(ascending=False)
for col in percentMissing[percentMissing>0.2].index:
    print(col)

member_id
orig_projected_additional_accrued_interest
hardship_payoff_balance_amount
payment_plan_start_date
hardship_last_payment_amount
hardship_end_date
hardship_length
hardship_loan_status
hardship_dpd
hardship_type
hardship_reason
hardship_start_date
hardship_status
hardship_amount
deferral_term
sec_app_mths_since_last_major_derog
settlement_percentage
settlement_term
debt_settlement_flag_date
settlement_amount
settlement_status
settlement_date
sec_app_revol_util
sec_app_mort_acc
sec_app_open_acc
sec_app_num_rev_accts
sec_app_earliest_cr_line
sec_app_open_act_il
sec_app_inq_last_6mths
sec_app_fico_range_high
sec_app_fico_range_low
sec_app_chargeoff_within_12_mths
sec_app_collections_12_mths_ex_med
revol_bal_joint
verification_status_joint
dti_joint
annual_inc_joint
desc
mths_since_last_record
mths_since_recent_bc_dlq
mths_since_last_major_derog
mths_since_recent_revol_delinq
next_pymnt_d
mths_since_last_delinq
il_util
mths_since_rcnt_il
open_rv_24m
max_bal_bc
all_util
open_rv_12m
i

In [57]:
data = data.drop(percentMissing[percentMissing>0.2].index, axis=1)

**ii) Impute remaining missing values:**

- Numerical columns: Use median (robust to outliers)
- Categorical columns: Use ‘Unknown’ as a new category

In [58]:
missingValues = data.isna().sum()
for col in missingValues[missingValues > 0].index:
    if data[col].dtype == object:
        data[col] = data[col].fillna("Unknown")
    else:
        data[col] = data[col].fillna(data[col].median())

### Drop Non-Informative Columns

In [59]:
pd.set_option('display.max_columns', None)

There's way too many values in these columns to be useful...

In [60]:
len(data["emp_title"].unique())

11770

In [61]:
len(data["addr_state"].unique())

51

In [62]:
data[["id", "loan_amnt", "url", "emp_title", "zip_code", "addr_state"]]

,id,loan_amnt,url,emp_title,zip_code,addr_state
0,39651438,32000.0,https://lendingclub.com/browse/loanDetail.acti...,Public Service,919xx,CA
1,16411620,9600.0,https://lendingclub.com/browse/loanDetail.acti...,Unknown,331xx,FL
2,45122316,4000.0,https://lendingclub.com/browse/loanDetail.acti...,System Analyst,333xx,FL
3,125356772,6025.0,https://lendingclub.com/browse/loanDetail.acti...,Admin assistant,021xx,MA
4,128490686,25000.0,https://lendingclub.com/browse/loanDetail.acti...,Coordinator,926xx,CA
...,...,...,...,...,...,...
22586,126862437,25000.0,https://lendingclub.com/browse/loanDetail.acti...,Director of New Play Development,602xx,IL
22587,143514465,1700.0,https://lendingclub.com/browse/loanDetail.acti...,Batch Ops Production Support Analyst,194xx,PA
22588,135390922,12000.0,https://lendingclub.com/browse/loanDetail.acti...,Assistant Sales Manager,770xx,TX
22589,70411635,16500.0,https://lendingclub.com/browse/loanDetail.acti...,Squad Leader,346xx,FL


None of these variables are very useful by themselves...

In [63]:
data = data.drop(["id", "loan_amnt", "url", "emp_title", "zip_code", "addr_state"], axis=1)

### Feature Engineering

**i) Date Features**


In [64]:
data["issue_d"]

0        Feb-2015
1        May-2014
2        Apr-2015
3        Dec-2017
4        Feb-2018
           ...   
22586    Jan-2018
22587    Nov-2018
22588    Jun-2018
22589    Feb-2016
22590    Dec-2015
Name: issue_d, Length: 22591, dtype: object

In [65]:
data["issuedDate"] = data["issue_d"].str.slice(-4).astype(int)

In [66]:
data["earliestCreditDate"] = data["earliest_cr_line"].str.slice(-4).astype(int)

In [67]:
data = data.drop(["issuedDate", "earliest_cr_line"], axis=1)

**ii) Term Feature**


In [68]:
data["termLength"] = data["term"].str.slice(1,4).astype(int)

**iii) High-Cardinality Categorical Features**


In [69]:
for col in data.select_dtypes("object").columns:
    print(col, len(data[col].unique()))

term 2
grade 7
sub_grade 35
emp_length 12
home_ownership 6
verification_status 3
issue_d 133
loan_status 8
pymnt_plan 2
purpose 14
title 1234
initial_list_status 2
application_type 2
hardship_flag 2
disbursement_method 2
debt_settlement_flag 2


These columns look problematic:

- `sub_grade`
- `issue_d`
- `title`

Let's just drop them...

In [70]:
data = data.drop(["sub_grade", "issue_d", "title"], axis=1)


**iv) One-Hot Encoding**

In [71]:
for col in data.select_dtypes("object").columns:
    data = pd.concat([data.drop(col, axis=1), pd.get_dummies(data[col], drop_first=True, prefix=col).astype(int)], axis=1)

### Filter Invalid Records.

In [72]:
data["int_rate"].describe()

count    22591.000000
mean        13.095236
std          4.981891
min        -11.547247
25%          9.440000
50%         12.620000
75%         15.990000
max         43.985799
Name: int_rate, dtype: float64

I don't think interest rate is supposed to be negative...

In [73]:
data = data[data["int_rate"] > 0]